In [7]:
# !pip install pandas
# !pip install datasets


In [8]:
import datasets

In [12]:
all_subset_names = ['aya_dataset', 'templated_afriqa', 'templated_afrisenti', 'templated_amharic_qa', 'templated_armenian_instruct', 'templated_bengali_news', 'templated_dutch_imdb', 'templated_hindi_headline', 'templated_hindi_news', 'templated_indic_paraphrase', 'templated_indic_sentiment', 'templated_indo_stories', 'templated_japanese_instruct', 'templated_joke_explaination', 'templated_ligurian_news', 'templated_masakhanews', 'templated_mintaka', 'templated_ntx_llm', 'templated_nusax_senti', 'templated_persian_farstail', 'templated_persian_instruct', 'templated_scirepeval', 'templated_seed_instruct', 'templated_soda', 'templated_tamil_stories', 'templated_tamil_thirukkural', 'templated_telugu_food', 'templated_telugu_jokes', 'templated_telugu_news', 'templated_telugu_poems', 'templated_telugu_riddles', 'templated_thai_pos', 'templated_thai_scb', 'templated_thai_usembassy', 'templated_thai_wikitionary', 'templated_turku_paraphrase', 'templated_ukranian_gec', 'templated_uner_llm', 'templated_urdu_news_category', 'templated_urdu_news_gen', 'templated_urdu_news_headline', 'templated_wiki_split', 'templated_xcsqa', 'templated_xlel_wd', 'templated_xwikis', 'translated_adversarial_qa', 'translated_cnn_dailymail', 'translated_dolly', 'translated_flan_coqa', 'translated_flan_cot', 'translated_flan_gem_wiki', 'translated_flan_lambada', 'translated_flan_qa', 'translated_hotpotqa', 'translated_joke_explaination', 'translated_mintaka', 'translated_mlqa', 'translated_nqopen', 'translated_paws', 'translated_piqa', 'translated_soda', 'translated_wiki_split', 'translated_wikiqa', 'translated_xlel_wd']


In [16]:
templated_subsets = [s for s in all_subset_names if "templated" in s]

In [43]:
translated_subsets = [s for s in all_subset_names if "translated" in s]

In [44]:
len(translated_subsets)

19

In [45]:
5000/19

263.1578947368421

In [18]:
from datasets import load_dataset


all_subset_names = ['aya_dataset', 'templated_afriqa', 'templated_afrisenti', 'templated_amharic_qa', 'templated_armenian_instruct', 'templated_bengali_news', 'templated_dutch_imdb', 'templated_hindi_headline', 'templated_hindi_news', 'templated_indic_paraphrase', 'templated_indic_sentiment', 'templated_indo_stories', 'templated_japanese_instruct', 'templated_joke_explaination', 'templated_ligurian_news', 'templated_masakhanews', 'templated_mintaka', 'templated_ntx_llm', 'templated_nusax_senti', 'templated_persian_farstail', 'templated_persian_instruct', 'templated_scirepeval', 'templated_seed_instruct', 'templated_soda', 'templated_tamil_stories', 'templated_tamil_thirukkural', 'templated_telugu_food', 'templated_telugu_jokes', 'templated_telugu_news', 'templated_telugu_poems', 'templated_telugu_riddles', 'templated_thai_pos', 'templated_thai_scb', 'templated_thai_usembassy', 'templated_thai_wikitionary', 'templated_turku_paraphrase', 'templated_ukranian_gec', 'templated_uner_llm', 'templated_urdu_news_category', 'templated_urdu_news_gen', 'templated_urdu_news_headline', 'templated_wiki_split', 'templated_xcsqa', 'templated_xlel_wd', 'templated_xwikis', 'translated_adversarial_qa', 'translated_cnn_dailymail', 'translated_dolly', 'translated_flan_coqa', 'translated_flan_cot', 'translated_flan_gem_wiki', 'translated_flan_lambada', 'translated_flan_qa', 'translated_hotpotqa', 'translated_joke_explaination', 'translated_mintaka', 'translated_mlqa', 'translated_nqopen', 'translated_paws', 'translated_piqa', 'translated_soda', 'translated_wiki_split', 'translated_wikiqa', 'translated_xlel_wd']


# Try to load the dataset info
dataset = load_dataset("CohereForAI/aya_collection", all_subset_names[0], split="train")


In [35]:
from datasets import load_dataset
import random
import os
import json
import gc  # For explicit garbage collection

def sample_from_dataset(subset_name, output_dir, num_samples=5000):
    """
    Sample from a single dataset subset and save to disk.
    """
    print(f"\nProcessing subset: {subset_name}")
    output_path = os.path.join(output_dir, f"{subset_name}_train_sample.json")
    
    try:
        # Load only the train split
        dataset = load_dataset("CohereForAI/aya_collection", subset_name, split="train")
        
        # Calculate indices for sampling
        total_size = len(dataset)
        if total_size < num_samples:
            print(f"Dataset has fewer than {num_samples} rows; sampling all {total_size} rows.")
            indices = range(total_size)
        else:
            indices = random.sample(range(total_size), num_samples)
        
        # Sample using indices
        sampled_data = [dataset[idx] for idx in indices]
        
        # Save to disk
        with open(output_path, "w", encoding='utf-8') as f:
            json.dump(sampled_data, f, ensure_ascii=False, indent=2)
        
        print(f"Successfully sampled {len(sampled_data)} entries from {subset_name}")
        print(f"Saved to: {output_path}")
        
    except Exception as e:
        print(f"Error processing {subset_name}: {str(e)}")
    
    finally:
        # Explicit cleanup
        if 'dataset' in locals():
            del dataset
        if 'sampled_data' in locals():
            del sampled_data
        gc.collect()  # Force garbage collection
        
# Create output directory
output_dir = "translated_subsets"
os.makedirs(output_dir, exist_ok=True)

# Process one subset at a time
# templated_subsets = ["aya_dataset"]
for subset_name in translated_subsets:
    sample_from_dataset(subset_name, output_dir)

print("\nAll subsets processed.")


Processing subset: aya_dataset
Successfully sampled 5000 entries from aya_dataset
Saved to: aya_sampled_data/aya_dataset_train_sample.json

All subsets processed.


In [36]:
import pandas as pd
import os
import json

# Directory containing the sampled JSON files
input_dir = "aya_sampled_data"

# List to store data from all JSONs
all_data = []

# Iterate through all JSON files in the directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".json"):
        subset_name = file_name.rsplit(".", 1)[0]  # Get the file name without the extension
        file_path = os.path.join(input_dir, file_name)
        
        # Load the JSON file
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            
        # Add a column for subset_name
        for entry in data:
            entry["subset_name"] = subset_name
        
        # Append to the all_data list
        all_data.extend(data)

# Create a pandas DataFrame from the combined data
df = pd.DataFrame(all_data)

In [37]:
# Save the DataFrame to a Parquet file without the index
output_file = "aya_sample_5k.parquet"
df.to_parquet(output_file, index=False)

print(f"DataFrame successfully saved to {output_file}")


DataFrame successfully saved to aya_sample_5k.parquet


In [38]:
len(df)

5000